In [1]:
from keras.models import Model
from keras.layers import concatenate
from keras.layers import Conv2D , DepthwiseConv2D , MaxPool2D , Input , GlobalAveragePooling2D ,AveragePooling2D, Dense , Dropout ,Activation, Flatten , BatchNormalization

def ComiaBlock(capa_prev, kernels, kernel_size, expansion_factor=1, stride=(1,1), padding="same", function="relu"):
    x = Conv2D(kernels * expansion_factor, kernel_size=kernel_size, padding=padding, strides=(1,1))(capa_prev)
    x = BatchNormalization()(x)
    x = Activation(function)(x)

    x = DepthwiseConv2D(kernel_size=kernel_size, padding=padding, strides=stride)(x)
    x = BatchNormalization()(x)
    x = Activation(function)(x)

    x = Conv2D(kernels, kernel_size=(1,1), padding=padding, strides=(1,1))(x)
    x = BatchNormalization()(x)

    if stride == (2,2) or capa_prev.shape[-1] != kernels:
        capa_prev = Conv2D(kernels, kernel_size=(1,1), padding=padding, strides=stride)(capa_prev)
        capa_prev = BatchNormalization()(capa_prev)

    x = concatenate([x, capa_prev])

    return x

def StemBlock(capa_prev):
    x = Conv2D(32, (3,3), strides=(2,2), padding='same')(capa_prev)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(32, (3,3), strides=(1,1), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(64, (3,3), strides=(1,1), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = MaxPool2D((3,3), strides=(2,2), padding='same')(x)

    return x

def ComiaBlock1(capa_prev):
    x = ComiaBlock(capa_prev=capa_prev, kernels=64, expansion_factor=2, kernel_size=(3,3), stride=(2,2))
    x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
    return x

def ComiaBlock2(capa_prev):
    rama1 = ComiaBlock(capa_prev=capa_prev, kernels= 96, expansion_factor=2, kernel_size=(3,3), stride=(1,1))
    rama1 = ComiaBlock(capa_prev=rama1, kernels= 96, expansion_factor=2, kernel_size=(3,3), stride=(1,1))
    rama1 = ComiaBlock(capa_prev=rama1, kernels= 128, expansion_factor=2, kernel_size=(3,3), stride=(1,1))

    rama2 = ComiaBlock(capa_prev=capa_prev, kernels= 96, expansion_factor=2, kernel_size=(3,3), stride=(1,1))
    rama2 = ComiaBlock(capa_prev=rama2, kernels= 128, expansion_factor=2, kernel_size=(3,3), stride=(1,1))
    rama2 = ComiaBlock(capa_prev=rama2, kernels= 128, expansion_factor=2, kernel_size=(3,3), stride=(1,1))
    
    rama3 = AveragePooling2D(pool_size=(3,3), strides=(1,1), padding='same' )(capa_prev)
    rama3 = ComiaBlock(capa_prev=rama3, kernels= 96, expansion_factor=2, kernel_size=(3,3), stride=(1,1))

    output = concatenate([rama1,rama2,rama3], axis = 3)

    return output

def ComiaBlock3(capa_prev):
    x = ComiaBlock(capa_prev=capa_prev, kernels=128, expansion_factor=2, kernel_size=(3,3), stride=(2,2))
    x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
    return x

def ComiaBlock4(capa_prev):
    rama1 = ComiaBlock(capa_prev=capa_prev, kernels= 128, expansion_factor=2, kernel_size=(3,3), stride=(1,1))
    rama1 = ComiaBlock(capa_prev=rama1, kernels= 128, expansion_factor=2, kernel_size=(3,3), stride=(1,1))
    rama1 = ComiaBlock(capa_prev=rama1, kernels= 192, expansion_factor=2, kernel_size=(3,3), stride=(1,1))
    rama1 = ComiaBlock(capa_prev=rama1, kernels= 256, expansion_factor=2, kernel_size=(3,3), stride=(1,1))

    rama2 = ComiaBlock(capa_prev=capa_prev, kernels= 64, expansion_factor=2, kernel_size=(3,3), stride=(1,1))
    rama2 = ComiaBlock(capa_prev=rama2, kernels= 64, expansion_factor=2, kernel_size=(3,3), stride=(1,1))
    rama2 = ComiaBlock(capa_prev=rama2, kernels= 128, expansion_factor=2, kernel_size=(3,3), stride=(1,1))
    
    rama3 = AveragePooling2D(pool_size=(3,3), strides=(1,1), padding='same' )(capa_prev)
    rama3 = ComiaBlock(capa_prev=rama3, kernels= 128, expansion_factor=2, kernel_size=(3,3), stride=(1,1))

    output = concatenate([rama1,rama2,rama3], axis = 3)

    return output

def ComiaBlock5(capa_prev):
    x = ComiaBlock(capa_prev=capa_prev, kernels=192, expansion_factor=2, kernel_size=(3,3), stride=(2,2))
    x = ComiaBlock(capa_prev=x, kernels=256, expansion_factor=3, kernel_size=(3,3), stride=(1,1))
    x = ComiaBlock(capa_prev=x, kernels=256, expansion_factor=3, kernel_size=(3,3), stride=(1,1))
    x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
    return x

def ComiaNet():

    input = Input(shape = (240,240,3))

    x = StemBlock(input)

    x = ComiaBlock1(x)
    x = ComiaBlock2(x)
    x = ComiaBlock3(x)
    x = ComiaBlock4(x)
    x = ComiaBlock5(x)

    model = Model(inputs=input, outputs=x, name = 'ComiaV1')
    
    return model

In [ ]:
from sklearn.model_selection import train_test_split
import os
import numpy as np

# Ruta a tus imágenes categorizadas por carpetas de clases
path = "C:/TODO/UNIVERSIDAD/TFG/food-101/images"

# Listar todas las carpetas de clases en el directorio
classes = [d.name for d in os.scandir(path) if d.is_dir()]
files = []
labels = []

# Recorrer cada carpeta de clase y recopilar los archivos de imágenes
for index, class_name in enumerate(classes):
    class_dir = os.path.join(path, class_name)
    class_files = [os.path.join(class_dir, f) for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]
    files.extend(class_files)
    labels.extend([index] * len(class_files))

# Convertir las listas en arrays de numpy para facilitar su manipulación
files = np.array(files)
labels = np.array(labels)

# Dividir los datos en conjuntos de entrenamiento y validación con muestreo estratificado
train_files, val_files, train_labels, val_labels = train_test_split(
    files, labels, test_size=0.1, random_state=42, stratify=labels
)


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Augmentation para el conjunto de entrenamiento
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.1  # Si necesitas subdividir aún más tus datos
)

# Solo reescalado para el conjunto de validación
val_datagen = ImageDataGenerator(
    rescale=1./255
)


In [3]:
import pandas as pd

# Crear DataFrames para los conjuntos de entrenamiento y validación
train_df = pd.DataFrame({
    'filename': train_files,
    'class': train_labels
})

val_df = pd.DataFrame({
    'filename': val_files,
    'class': val_labels
})

# Crear un mapeo de índices numéricos a nombres de clase
index_to_class = {i: name for i, name in enumerate(classes)}

# Convertir índices numéricos a nombres de clase en los DataFrames
train_df['class'] = train_df['class'].map(index_to_class)
val_df['class'] = val_df['class'].map(index_to_class)


# Crear los generadores
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='filename',
    y_col='class',
    target_size=(240, 240),
    batch_size=16,
    class_mode='categorical',
    shuffle=True
)

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col='filename',
    y_col='class',
    target_size=(240, 240),
    batch_size=16,
    class_mode='categorical',
    shuffle=False
)


Found 90900 validated image filenames belonging to 101 classes.

Found 10100 validated image filenames belonging to 101 classes.


In [ ]:
from comIA import ComiaNet
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.models import Model
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau

# Crear el modelo base de EfficientNetB0
base_model = ComiaNet()
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(len(classes), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

# Compilación del modelo
from keras.optimizers import Adam
opt=Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
# Entrenamiento
history = model.fit(
    train_generator,
    epochs=30,
    validation_data=val_generator,
    verbose=1,
    callbacks = [reduce_lr]
)


In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

epochs = range(1, len(acc) + 1)

# Crear gráfico
plt.figure(figsize=(10, 6))
plt.plot(epochs, acc, 'b', label='Precisión en entrenamiento')  # Azul y línea continua
plt.plot(epochs, val_acc, 'g', label='Precisión en validación')  # Verde y línea continua

# Encontrar y plotear los puntos máximos
max_acc_index = acc.index(max(acc))
max_val_acc_index = val_acc.index(max(val_acc))
plt.scatter([epochs[max_acc_index]], [acc[max_acc_index]], color='red', s=100, label='Máximo en entrenamiento')
plt.scatter([epochs[max_val_acc_index]], [val_acc[max_val_acc_index]], color='red', s=100, label='Máximo en validación')

plt.title('Precisión durante el entrenamiento y la validación')
plt.xlabel('Epochs')
plt.ylabel('Precisión')
plt.legend()

plt.show()

